# 猫狗大战 毕业项目——迁移NASNetMobile

## 开始
导入一切并我们设置所使用的GPU。
- dev1：K40c

In [1]:
%matplotlib inline
#import utilities
import os
import shutil
import numpy as np
import random
from tqdm import tqdm  
from time import time
from PIL import Image
import h5py
import pandas as pd
from helper import *

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *
from keras import backend as K

#如果系统上有多块GPU，“0”可以替换成其它GPU的编号
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/pengjun/.conda/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 数据文件处理
训练数据包括12500张猫的图片和12500张狗的图片。我们为数据文件建立symbol link并划分为训练集和验证集，所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

In [6]:
#为数据连理symbol-link
train_data_dir, valid_data_dir, test_data_dir = prepare_data_file()

100%|██████████| 12500/12500 [00:00<00:00, 183339.28it/s]


In [2]:
#导入训练数据和测试数据
X_train, Y_train, X_test = load_feature_data("feature_NASNet.h5")

In [3]:
#构造模型并显示所有网络层的名称
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [4]:
#训练模型并导出权重参数
filepath="nasnet-tune0-best_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 1s 33us/step - loss: 0.2525 - acc: 0.8925 - val_loss: 0.1472 - val_acc: 0.9434

Epoch 00001: val_loss improved from inf to 0.14718, saving model to nasnet-tune0-best_weight.h5
Epoch 2/20
20000/20000 [==============================] - 0s 16us/step - loss: 0.1531 - acc: 0.9358 - val_loss: 0.1340 - val_acc: 0.9418

Epoch 00002: val_loss improved from 0.14718 to 0.13405, saving model to nasnet-tune0-best_weight.h5
Epoch 3/20
20000/20000 [==============================] - 0s 17us/step - loss: 0.1375 - acc: 0.9408 - val_loss: 0.1248 - val_acc: 0.9444

Epoch 00003: val_loss improved from 0.13405 to 0.12478, saving model to nasnet-tune0-best_weight.h5
Epoch 4/20
20000/20000 [==============================] - 0s 23us/step - loss: 0.1329 - acc: 0.9443 - val_loss: 0.1139 - val_acc: 0.9522

Epoch 00004: val_loss improved from 0.12478 to 0.11393, saving model to nasnet-tune0-best_weight.h5
Epo

In [7]:
#在测试集上进行预测并导出预测值
predict_on_model(test_data_dir, X_test, model, "pred-nasnet-tune0.csv")

12500/12500 [==============================] - 0s 25us/step
Found 12500 images belonging to 1 classes.


/home/pengjun/DLND/Cat_vs_Dog/helper.py:130: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(index-1, 'label', y_test[i])


> 这一模型在Kaggle上的得分为0.12040